In [31]:
!pip install -q sec-api

In [32]:
API_KEY = 'f2a13f97afd2f1f12f7e1f74e0e35b178e6b00da0b08702b0a25cbaa8f1b2a3a'

In [33]:
from sec_api import QueryApi

queryApi = QueryApi(api_key=API_KEY)

In [34]:
import pandas as pd


In [35]:
def get_filings(query):
  from_param = 0
  size_param = 200
  all_filings = []

  while True:
    query['from'] = from_param
    query['size'] = size_param

    response = queryApi.get_filings(query)
    filings = response['filings']

    if len(filings) == 0:
      break

    all_filings.extend(filings)

    from_param += size_param

  return pd.json_normalize(all_filings)


form_type_query  = 'formType:("10-K") AND NOT formType:("10-K/A", NT)'
ticker_query     = 'ticker:(MSFT, TSLA, AAPL)'
date_range_query = 'filedAt:[2022-01-01 TO 2024-12-31]'

lucene_query = f"{form_type_query} AND {ticker_query} AND {date_range_query}"

search_query = {
  "query": lucene_query,
  "from": "0",
  "size": "200",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = get_filings(search_query)

In [36]:
filings.head()

,ticker,formType,accessionNo,cik,companyNameLong,companyName,linkToFilingDetails,description,linkToTxt,filedAt,documentFormatFiles,periodOfReport,entities,id,seriesAndClassesContractsInformation,linkToHtml,linkToXbrl,dataFiles
0,AAPL,10-K,0000320193-24-000123,320193,Apple Inc. (Filer),Apple Inc.,https://www.sec.gov/Archives/edgar/data/320193...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/320193...,2024-11-01T06:01:36-04:00,"[{'sequence': '1', 'size': '1503780', 'documen...",2024-09-28,"[{'fiscalYearEnd': '0928', 'stateOfIncorporati...",de98dc75c98b80c2bbfeb8d07503354b,[],https://www.sec.gov/Archives/edgar/data/320193...,,"[{'sequence': '14', 'size': '58200', 'document..."
1,MSFT,10-K,0000950170-24-087843,789019,MICROSOFT CORP (Filer),MICROSOFT CORP,https://www.sec.gov/Archives/edgar/data/789019...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/789019...,2024-07-30T16:06:22-04:00,"[{'sequence': '1', 'size': '6860911', 'documen...",2024-06-30,"[{'fiscalYearEnd': '0630', 'stateOfIncorporati...",b80c727c9b0c7ef7fb3c6f15b3f6f3d1,[],https://www.sec.gov/Archives/edgar/data/789019...,,"[{'sequence': '14', 'size': '2099152', 'docume..."
2,TSLA,10-K,0001628280-24-002390,1318605,"Tesla, Inc. (Filer)","Tesla, Inc.",https://www.sec.gov/Archives/edgar/data/131860...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/131860...,2024-01-26T21:00:20-05:00,"[{'sequence': '1', 'size': '2672746', 'documen...",2023-12-31,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...",72ccf726af50ba95c163db42643b93ab,[],https://www.sec.gov/Archives/edgar/data/131860...,,"[{'sequence': '8', 'size': '104440', 'document..."
3,AAPL,10-K,0000320193-23-000106,320193,Apple Inc. (Filer),Apple Inc.,https://www.sec.gov/Archives/edgar/data/320193...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/320193...,2023-11-02T18:08:27-04:00,"[{'sequence': '1', 'size': '1558924', 'documen...",2023-09-30,"[{'fiscalYearEnd': '0930', 'stateOfIncorporati...",0de409aa5047085970060a9efa218f8b,[],https://www.sec.gov/Archives/edgar/data/320193...,,"[{'sequence': '8', 'size': '59730', 'documentU..."
4,MSFT,10-K,0000950170-23-035122,789019,MICROSOFT CORP (Filer),MICROSOFT CORP,https://www.sec.gov/Archives/edgar/data/789019...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/789019...,2023-07-27T16:01:56-04:00,"[{'sequence': '1', 'size': '9963591', 'documen...",2023-06-30,"[{'fiscalYearEnd': '0630', 'stateOfIncorporati...",c5157b39d2e3e4516ceade25a58f1b85,[],https://www.sec.gov/Archives/edgar/data/789019...,,"[{'sequence': '8', 'size': '389706', 'document..."


In [37]:
urls = filings[['ticker',
                'formType',
                'periodOfReport',
                'filedAt',
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(10)

,ticker,formType,periodOfReport,filedAt,filingUrl,financialReportsUrl
0,AAPL,10-K,2024-09-28,2024-11-01T06:01:36-04:00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,MSFT,10-K,2024-06-30,2024-07-30T16:06:22-04:00,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
2,TSLA,10-K,2023-12-31,2024-01-26T21:00:20-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
3,AAPL,10-K,2023-09-30,2023-11-02T18:08:27-04:00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,MSFT,10-K,2023-06-30,2023-07-27T16:01:56-04:00,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
5,TSLA,10-K,2022-12-31,2023-01-30T21:29:15-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
6,AAPL,10-K,2022-09-24,2022-10-27T18:01:14-04:00,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
7,MSFT,10-K,2022-06-30,2022-07-28T16:06:19-04:00,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
8,TSLA,10-K,2021-12-31,2022-02-04T20:11:27-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...


In [38]:
!pip install -q pandarallel

In [39]:
import os, requests

folder_path = './reports'

if not os.path.exists(folder_path):
  os.makedirs(folder_path)


def download_report(filing):
  reports_path = filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')
  base_url = 'https://archive.sec-api.io/' + reports_path
  render_api_url = base_url + '?token=' + API_KEY

  response = requests.get(render_api_url)

  file_name = f"{filing['ticker']}-{filing['periodOfReport']}-{filing['formType']}.xlsx"
  file_path = f"{folder_path}/{file_name}"

  output = open(file_path, 'wb')
  output.write(response.content)
  output.close()

In [40]:
from pandarallel import pandarallel

number_of_workers = 4
pandarallel.initialize(progress_bar=True, nb_workers=number_of_workers, verbose=0)
urls.parallel_apply(download_report, axis=1)
print(f"✅ Downloaded {len(urls)} reports")

✅ Downloaded 9 reports
